In [12]:
import random
import pandas as pd
from sage.graphs.graph_generators import graphs  # Uvoz SageMath grafov

grafi = []  # Seznam za shranjevanje grafov

# Zanka za generiranje grafov za število vozlišč od 1 do 9
for i in range(1, 9):
    for G in graphs.nauty_geng(f'{i} -c'):  # 'c' pomeni povezane grafe
        # Dodaj graf G v seznam grafov
        grafi.append(G)


###########################################################

def SDCTD_stevilo2(g):
    h = g.complement()

    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    p.set_objective(sum([x[v] for v in g]))

    for v in g.vertices():
        neighbors = g.neighbors(v)  
        if neighbors:
            p.add_constraint(sum((x[w] + x[v]) for w in neighbors) >= 1)
        else:
            p.add_constraint(x[v] == 1)
            
    zaustavi = False

    for v in g.vertices():
        neighbors_complement = h.neighbors(v)  
        if neighbors_complement:  
            p.add_constraint(sum(x[w] for w in neighbors_complement) >= 1)

        else: 
            zaustavi = True
            break 

    if not zaustavi == True:
        p.solve()
        x = p.get_values(x)
        return(sum(1 for i in x.values() if i == 1))
    else:
        return None
    
############################################################ 

rezultati = []

for n in range(1, 1000):
    # Izbira dveh grafov naključno iz seznama grafov
    g1 = random.choice(grafi)
    g2 = random.choice(grafi)
    # Kartezijski produkt med g1 in g2
    g3 = g1.cartesian_product(g2)

    # Izračun dominacijskega števila za vsak graf
    dominacijsko_stevilo_graf1 = SDCTD_stevilo2(g1)  # Predpostavljamo, da imate to funkcijo definirano
    dominacijsko_stevilo_graf2 = SDCTD_stevilo2(g2)
    dominacijsko_stevilo_graf3 = SDCTD_stevilo2(g3)
    
    # Dodajanje rezultatov v seznam
    rezultati.append({
        'Izbira grafov': f'Izbira {n}',
        'SDCTD_graf1': dominacijsko_stevilo_graf1,
        'SDCTD_graf2': dominacijsko_stevilo_graf2,
        'SDCTD_kartezicni': dominacijsko_stevilo_graf3
    })
    
pd.set_option('display.max_rows', None)
# Ustvari pandas DataFrame iz seznama rezultatov
df = pd.DataFrame(rezultati)


def vizingova_domneva(row):
    if isinstance(row['SDCTD_graf1'], (int, float)) and isinstance(row['SDCTD_graf2'], (int, float)) and isinstance(row['SDCTD_kartezicni'], (int, float)):
        # Preveri, ali je SDCTD_graf1 * SDCTD_graf2 > SDCTD_kartezicni
        if row['SDCTD_graf1'] * row['SDCTD_graf2'] > row['SDCTD_kartezicni']:
            return False
        else:
            return True
    else: return True


# Dodaj stolpec 'Vizingova domneva'
df['Vizingova domneva'] = df.apply(vizingova_domneva, axis=1)

# Izpiši DataFrame
print(df)

    Izbira grafov SDCTD_graf1 SDCTD_graf2 SDCTD_kartezicni  Vizingova domneva
0        Izbira 1           3           3                7               True
1        Izbira 2           3           2               10               True
2        Izbira 3           4           3                8               True
3        Izbira 4           3           3               10               True
4        Izbira 5           3        None                6               True
5        Izbira 6           3           3               11               True
6        Izbira 7           3           2                9               True
7        Izbira 8           3           2                8               True
8        Izbira 9           3           2                9               True
9       Izbira 10           3           3                8               True
10      Izbira 11           3           3               10               True
11      Izbira 12           2           2               10      